<a href="https://colab.research.google.com/github/biswa2005/ML-Learning/blob/main/Model_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = sns.load_dataset('iris')
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [3]:
df['species'].value_counts()

,count
species,
setosa,50
versicolor,50
virginica,50


In [5]:
X = df.drop('species', axis=1)
y = df['species']

In [10]:
from sklearn.svm import SVC

model_svm = SVC(gamma = 'auto')

###GridSearchCV

In [11]:
from sklearn.model_selection import GridSearchCV

In [12]:
classifier = GridSearchCV((model_svm), {
    'C' : [1,10,20,30],
    'kernel' : ['rbf', 'linear'],
}, cv=5, return_train_score=False)

In [13]:
classifier.fit(X, y)

GridSearchCV(cv=5, estimator=SVC(gamma='auto'),
             param_grid={'C': [1, 10, 20, 30], 'kernel': ['rbf', 'linear']})

In [14]:
classifier.cv_results_

{'mean_fit_time': array([0.00683889, 0.00429831, 0.00934396, 0.00396938, 0.00506291,
        0.0042151 , 0.00249567, 0.0023932 ]),
 'std_fit_time': array([0.00206864, 0.00011849, 0.00152503, 0.0001952 , 0.00226165,
        0.00302581, 0.00016975, 0.00016852]),
 'mean_score_time': array([0.00621572, 0.00314064, 0.00560937, 0.00291228, 0.00446754,
        0.00283756, 0.00188856, 0.00173402]),
 'std_score_time': array([2.10056937e-03, 2.34555389e-04, 1.37721097e-03, 1.54559099e-04,
        2.56444864e-03, 1.71010659e-03, 2.56481900e-04, 4.71145517e-05]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20, 30, 30],
              mask=[False, False, False, False, False, False, False, False],
        fill_value=999999),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear',
                    'rbf', 'linear'],
              mask=[False, False, False, False, False, False, False, False],
        fill_value=np.str_('?'),
             dtype=object),
 'params': [

In [15]:
results = pd.DataFrame(classifier.cv_results_)

In [16]:
results[['param_C','param_kernel','mean_test_score']]


,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667
6,30,rbf,0.960000
7,30,linear,0.960000


###RandomizedSearchCV

In [26]:
from sklearn.model_selection import RandomizedSearchCV

In [27]:
Random_classifier = RandomizedSearchCV((model_svm), {
    'C' : [1,10,20,30],
    'kernel' : ['rbf', 'linear'],
}, n_iter=4, cv=5, return_train_score=False)

In [28]:
Random_classifier.fit(X, y)

RandomizedSearchCV(cv=5, estimator=SVC(gamma='auto'), n_iter=4,
                   param_distributions={'C': [1, 10, 20, 30],
                                        'kernel': ['rbf', 'linear']})

In [29]:
Random_classifier.cv_results_

{'mean_fit_time': array([0.00404701, 0.00479574, 0.00362477, 0.00399871]),
 'std_fit_time': array([0.00029548, 0.00127277, 0.00056743, 0.00113512]),
 'mean_score_time': array([0.00389676, 0.00333791, 0.00257349, 0.00274701]),
 'std_score_time': array([0.00135458, 0.00100187, 0.00032144, 0.00079867]),
 'param_kernel': masked_array(data=['rbf', 'rbf', 'rbf', 'linear'],
              mask=[False, False, False, False],
        fill_value=np.str_('?'),
             dtype=object),
 'param_C': masked_array(data=[10, 1, 20, 10],
              mask=[False, False, False, False],
        fill_value=999999),
 'params': [{'kernel': 'rbf', 'C': 10},
  {'kernel': 'rbf', 'C': 1},
  {'kernel': 'rbf', 'C': 20},
  {'kernel': 'linear', 'C': 10}],
 'split0_test_score': array([0.96666667, 0.96666667, 0.96666667, 1.        ]),
 'split1_test_score': array([1., 1., 1., 1.]),
 'split2_test_score': array([0.96666667, 0.96666667, 0.9       , 0.9       ]),
 'split3_test_score': array([0.96666667, 0.96666667, 0.966

In [30]:
results = pd.DataFrame(Random_classifier.cv_results_)

In [31]:
results[['param_C','param_kernel','mean_test_score']]


,param_C,param_kernel,mean_test_score
0,10,rbf,0.980000
1,1,rbf,0.980000
2,20,rbf,0.966667
3,10,linear,0.973333
